In [32]:
import pandas as pd
import os
from transformers import BertTokenizer
import torch

In [ ]:
"""dictionary"""
DICT_PATH = "./dataset/TC/Lexicon2003-72k.txt"
df_dict = pd.read_csv(DICT_PATH, encoding="cp950", header=None)

In [ ]:
Human_N = '1'
Percent = {10:'01', 20:'02', 30:'03'}
FILE_NAME = "PTSND20011107_1"
ANS_PATH = "./dataset/TC/TestFile_TD_DOS/DocRatio/Human_" + Human_N + "/" + Percent[10] + "/" + FILE_NAME + ".txt"
ans_file = open(ANS_PATH)
raw_ans = ans_file.read()

In [ ]:
raw_ans = raw_ans.replace("a","").replace("\n", "")
ans = raw_ans.split(" ")
ans = ans[:-1]

In [ ]:
"""
answer decoder
input: index sequence (list of string)
output: decode of index (string)
"""
def decoder(index):
    result = ""
    for i in index:
        result += df_dict.loc[int(i), 0]
#     print("f",result)
    return result
ans = decoder(ans)
# ans

In [ ]:
NEWS_PATH = "./dataset/TC/TestFile_TD_DOS/" + FILE_NAME + ".txt"
news_file = open(NEWS_PATH, encoding="cp950")
news = news_file.read()

In [ ]:
"""
content preprocess
input: content (string), answer (string)
output: processed content(string), label (list of 1/0), cls id (list of int),
        origin content (list of string), target text (list of string)
"""
def content_preprocess(content, ans):
    content = content.replace(" ", "")
    content_list = content.splitlines()
    origin_content = content_list[:] # create a shallow copy, if don't add [:] will get the pointer
    result_label = []
    segments_ids = []
    cls_ids = []
    tgt_text_list = []
    pos = 0
    for i, text in enumerate(content_list):
        if (pos < 512):
            cls_ids.append(pos)
            pos += len(text) + 2 # +2 for [CLS], [SEP]
            if (ans.find(text) != -1):
                result_label.append(1) # label
                tgt_text_list.append(text)
            else:
                result_label.append(0) # label
        content_list[i] = "[CLS] " + text + " [SEP]"
    result_content = ' '.join(content_list)
    tgt_text = '<p>'.join(tgt_text_list)
    return result_content, result_label, cls_ids, origin_content, tgt_text
content, label, cls_ids, origin_content, tgt_text = content_preprocess(news, ans)
print(label)
print(cls_ids)
print(origin_content)
print(tgt_text)
content

In [ ]:
print(len(label))
print(len(cls_ids))

In [ ]:
from transformers import BertTokenizer
LM = "hfl/chinese-bert-wwm"
tokenizer = BertTokenizer.from_pretrained(LM)



In [41]:
bert_dict = tokenizer.encode_plus(content,
                                         add_special_tokens = False,
                                         return_token_type_ids = False,
                                         max_length=512,
                                         pad_to_max_length=True,
                                         return_tensors='pt',
                                         truncation=True)
# print(bert_dict)

In [ ]:
"""
segement embedding
code from BERTSUM
input: input_ids (list of int)
output: segment embedding (list of int)
"""
def create_segment(input_ids):
    segments_ids = []
    _segs = [-1] + [i for i, t in enumerate(input_ids) if t == tokenizer.vocab['[SEP]']]
    segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
    for i, s in enumerate(segs):
        if (i % 2 == 0):
            segments_ids += s* [0]
        else:
            segments_ids += s * [1]
    return segments_ids
bert_dict['input_ids'][0][511] = tokenizer.vocab['[SEP]']
segments_ids = create_segment(bert_dict['input_ids'][0])
# print(len(bert_dict['input_ids'][0]))
# print(len(segments_ids))

In [ ]:
data_dict = {"src": bert_dict['input_ids'][0].tolist(), "labels": label, "segs": segments_ids, 'clss': cls_ids,
                       'src_txt': origin_content, "tgt_txt": tgt_text}
# print(data_dict)

In [ ]:
import torch
dataset = []
dataset.append(data_dict)
torch.save(dataset, "simple_test.train.0.bert.pt")

In [ ]:
temp = torch.load("simple_test.train.0.bert.pt")
print(len(temp))

In [ ]:
print(len(temp[0]['labels']))

---

In [16]:
import torch
# temp = torch.load("../BertSum/bert_data/cnndm.train.0.bert.pt")
temp = torch.load("./dataset/bert_data_for_BERTSUM/TC/PTS.train.0.bert.pt")
# temp = torch.load("./simple_test.train.0.bert.pt")

In [17]:
r = set()
for i, e in enumerate(temp):
    if (len(e['src']) != 512):
        print(i, " src != 512")
        r.add(i)
    elif (len(e['labels']) != len(e['clss'])):
        print(i, " labels != clss")
        r.add(i)
    elif (len(e['segs']) != 512):
        print(i, " segs != 512")
        r.add(i)
r

set()

In [15]:
print(len(temp[0]['clss']))

16
